## Pré-processing sur df_train et df_test de Data_preprocessing_resampling
Filtrage stop Words / Tokenization / Lemmatisation



Accès au dossier du projet sur Google Drive

In [ ]:
import sys
import os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive/', force_remount=True)
    %cd drive/MyDrive/nov23_bds_rakuten
else:
    root_files = os.listdir()
    if not "requirements.txt" in root_files:
        print("The jupyter server root should be set to root of git project")
        print("If you're on vscode add the following to your settings.json file")
        print('"jupyter.notebookFileRoot": "${workspaceFolder}"')

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1_2JJeYhhAHn4xWGD5kC2VtjM-_SxznEm/nov23_bds_rakuten


# Importation des librairies

In [ ]:
import pandas as pd
#from nltk.tokenize import PunktSentenceTokenizer
#from nltk.tokenize import word_tokenize
#from wordcloud import WordCloud #, STOPWORDS

import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize.regexp import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

# Télécharger les ressources de nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
nltk.download('stopwords')



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Chargement du fichier source \data\output\preprocessing-lang\X_train_pre.csv

In [ ]:
df_train = pd.read_csv('output/data-preprocessing-resampling/df_train.csv', index_col=0)
display(df_train.head(2))
df_test = pd.read_csv('output/data-preprocessing-resampling/df_test.csv', index_col=0)
display(df_test.head(2))

,productid,imageid,text,lang,text_fr,imagefile,prdtypecode
0,131641431,885888766,Jesus - Cahiers Du Libre Avenir - Prêtre autre...,fr,Jesus - Cahiers Du Libre Avenir - Prêtre autre...,10/image_885888766_product_131641431.jpg,10
1,745789761,1304710852,The Inglorious Arts Of Peace: Exhibitions In C...,en,Les arts de la paix sans gloire: expositions d...,10/image_1304710852_product_745789761.jpg,10


,productid,imageid,text,lang,text_fr,imagefile,prdtypecode
index,,,,,,,
29927,3469563513,1215209457,Adventure Parks Heavy Duty Tree Strap - Dont t...,en,Les parcs d'aventure STRAP DURT DE DURG - N'at...,1280/image_1215209457_product_3469563513.jpg,1280
42596,4152889175,1309607798,Magideal 2 Pcs Orff Instrument De Musique Hoch...,fr,Magideal 2 Pcs Orff Instrument De Musique Hoch...,1320/image_1309607798_product_4152889175.jpg,1320


# Filtrage stop Words et Tokenization du texte

Envi

In [ ]:
# Création d'une liste personnalisée de stopwords français, anglais et allemends
french_stopwords = set(nltk_stopwords.words('french'))
english_stopwords = set(nltk_stopwords.words('english'))
dutch_stopwords = set(nltk_stopwords.words('dutch'))
combined_stopwords = french_stopwords.union(english_stopwords)
combined_stopwords = combined_stopwords.union(dutch_stopwords)

# Fonction pour nettoyer et extraire les mots, en excluant les stopwords
def extract_words(text):
    # Utilisation d'une expression régulière pour ne conserver que les mots
    words = re.findall(r'\w+', text.lower())
    # Filtrage des mots en excluant les stopwords, les chiffres et les mots de moins de 2 lettres, et suppression des doublons
    unique_words = {word for word in words if word not in combined_stopwords and not word.isdigit() and len(word) > 1}
    return list(unique_words)

# Ajout colonne mots_text_fr contenant les mots de la colonne texte
df_train['mots_text_fr'] = df_train['text_fr'].apply(extract_words)

# Affichage du résultat
display(df_train.head())
display(df_train['text_fr'][0])
df_train['mots_text_fr'][0]

,productid,imageid,text,lang,text_fr,imagefile,prdtypecode,mots_text_fr
0,131641431,885888766,Jesus - Cahiers Du Libre Avenir - Prêtre autre...,fr,Jesus - Cahiers Du Libre Avenir - Prêtre autre...,10/image_885888766_product_131641431.jpg,10,"[avenir, autrement, jesus, prêtre, cahiers, li..."
1,745789761,1304710852,The Inglorious Arts Of Peace: Exhibitions In C...,en,Les arts de la paix sans gloire: expositions d...,10/image_1304710852_product_745789761.jpg,10,"[canadienne, sans, xixe, paix, gloire, exposit..."
2,651353882,1033554185,Lecons De Sciences - Cours Superieur - Premier...,fr,Lecons De Sciences - Cours Superieur - Premier...,10/image_1033554185_product_651353882.jpg,10,"[sciences, annee, cours, premiere, lecons, sup..."
3,4043473264,1306457352,Generations Of Women Historians -,en,Générations de femmes historiens -,10/image_1306457352_product_4043473264.jpg,10,"[historiens, femmes, générations]"
4,679913105,1034656535,Oeuvres Dramatiques De F. Schiller Tome 6 -,fr,Oeuvres Dramatiques De F. Schiller Tome 6 -,10/image_1034656535_product_679913105.jpg,10,"[tome, dramatiques, oeuvres, schiller]"


'Jesus - Cahiers Du Libre Avenir - Prêtre autrement.'

['avenir', 'autrement', 'jesus', 'prêtre', 'cahiers', 'libre']

In [ ]:
# Ajout colonne mots_text_fr contenant les mots de la colonne texte
df_test['mots_text_fr'] = df_test['text_fr'].apply(extract_words)

# Affichage du résultat
display(df_test.head())

,productid,imageid,text,lang,text_fr,imagefile,prdtypecode,mots_text_fr
index,,,,,,,,
29927,3469563513,1215209457,Adventure Parks Heavy Duty Tree Strap - Dont t...,en,Les parcs d'aventure STRAP DURT DE DURG - N'at...,1280/image_1215209457_product_3469563513.jpg,1280,"[attachez, pouvez, faire, correctement, balanç..."
42596,4152889175,1309607798,Magideal 2 Pcs Orff Instrument De Musique Hoch...,fr,Magideal 2 Pcs Orff Instrument De Musique Hoch...,1320/image_1309607798_product_4152889175.jpg,1320,"[enfants, cm, stimuler, développant, orff, acc..."
158,1190507078,1068211845,Casa Padrino Baroque Dining Room With Armrests...,en,Casa Padrino Baroque Dining Room avec des acco...,1560/image_1068211845_product_1190507078.jpg,1560,"[pouvez, cm, produit, original, hauteur, motif..."
20061,4236041996,1324088536,T6 Xml Zoomable Led Lampe De Poche Tactique + ...,fr,T6 Xml Zoomable Led Lampe De Poche Tactique + ...,1302/image_1324088536_product_4236041996.jpg,1302,"[clickyswitch, emitter, type, tactique, 18650m..."
20931,3836739131,1250952042,Organisateur Enfants En Bois Keepsake Cadeau P...,fr,Organisateur Enfants En Bois Keepsake Cadeau P...,1280/image_1250952042_product_3836739131.jpg,1280,"[stockage, temps, non, séparée, venir, placez,..."


# Lemmatisation

Fonction pour convertir les tags POS de nltk à ceux compatibles avec WordNetLemmatizer (merci chatGPT)

Fonction qui permet de déterminer s'il faut lemmatiser un verbe, un nom,...

Préparation du lemmatizer

In [ ]:
# Fonction pour convertir les tags POS de nltk à ceux compatibles avec WordNetLemmatizer
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
# Initialisation du lemmatizer
lemmatizer = WordNetLemmatizer()

# Fonction pour lemmatiser une liste de mots
def lemmatize_words(words):
    return [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in words]

Application de la fonction de lemmatisation à la colonne 'mots_text'. ATTENTION CODE > 15 min

In [ ]:
df_train['mots_lemmatises'] = df_train['mots_text_fr'].apply(lemmatize_words)

In [ ]:
df_test['mots_lemmatises'] = df_test['mots_text_fr'].apply(lemmatize_words)

Affichage du DataFrame résultant

In [ ]:
print(df_train.head(5))

    productid     imageid                                               text  \
0   131641431   885888766  Jesus - Cahiers Du Libre Avenir - Prêtre autre...   
1   745789761  1304710852  The Inglorious Arts Of Peace: Exhibitions In C...   
2   651353882  1033554185  Lecons De Sciences - Cours Superieur - Premier...   
3  4043473264  1306457352                 Generations Of Women Historians -    
4   679913105  1034656535       Oeuvres Dramatiques De F. Schiller Tome 6 -    

  lang                                            text_fr  \
0   fr  Jesus - Cahiers Du Libre Avenir - Prêtre autre...   
1   en  Les arts de la paix sans gloire: expositions d...   
2   fr  Lecons De Sciences - Cours Superieur - Premier...   
3   en                 Générations de femmes historiens -   
4   fr       Oeuvres Dramatiques De F. Schiller Tome 6 -    

                                    imagefile  prdtypecode  \
0    10/image_885888766_product_131641431.jpg           10   
1   10/image_1304710852_prod

La colonne mots_lemmatises est d'abord convertie en une colonne texte_lemmatise où chaque liste de mots est transformée en une chaîne de caractères.

In [ ]:
# Convertir les listes de mots en chaînes de caractères
df_train['texte_lemmatise'] = df_train['mots_lemmatises'].apply(lambda x: ' '.join(x))
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109566 entries, 0 to 109565
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   productid        109566 non-null  int64 
 1   imageid          109566 non-null  int64 
 2   text             109566 non-null  object
 3   lang             109566 non-null  object
 4   text_fr          109566 non-null  object
 5   imagefile        109566 non-null  object
 6   prdtypecode      109566 non-null  int64 
 7   mots_text_fr     109566 non-null  object
 8   mots_lemmatises  109566 non-null  object
 9   texte_lemmatise  109566 non-null  object
dtypes: int64(3), object(7)
memory usage: 13.2+ MB


In [ ]:
df_test['texte_lemmatise'] = df_test['mots_lemmatises'].apply(lambda x: ' '.join(x))

# Suppression des colonnes superflues

In [ ]:
X_train = df_train.drop(['text_fr', 'mots_text_fr', 'mots_lemmatises'],axis=1)
X_train.rename(columns={'texte_lemmatise': 'text_fr'}, inplace=True)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109566 entries, 0 to 109565
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   productid    109566 non-null  int64 
 1   imageid      109566 non-null  int64 
 2   text         109566 non-null  object
 3   lang         109566 non-null  object
 4   imagefile    109566 non-null  object
 5   prdtypecode  109566 non-null  int64 
 6   text_fr      109566 non-null  object
dtypes: int64(3), object(4)
memory usage: 10.7+ MB


In [ ]:
X_test = df_test.drop(['text_fr', 'mots_text_fr', 'mots_lemmatises'],axis=1)
X_test.rename(columns={'texte_lemmatise': 'text_fr'}, inplace=True)
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8492 entries, 29927 to 18545
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   productid    8492 non-null   int64 
 1   imageid      8492 non-null   int64 
 2   text         8492 non-null   object
 3   lang         8492 non-null   object
 4   imagefile    8492 non-null   object
 5   prdtypecode  8492 non-null   int64 
 6   text_fr      8492 non-null   object
dtypes: int64(3), object(4)
memory usage: 788.8+ KB


# Export preprocessed dataframe

In [ ]:
# Identifier les lignes où la colonne 'texte' contient NaN
lignes_avec_nan = X_train['text_fr'].isna()

# Afficher les lignes avec NaN
print("\nLignes avec NaN dans la colonne 'texte':")
print(X_train.loc[lignes_avec_nan])


Lignes avec NaN dans la colonne 'texte':
Empty DataFrame
Columns: [productid, imageid, text, lang, imagefile, prdtypecode, text_fr]
Index: []


In [ ]:
# Identifier les lignes où la colonne 'texte' contient NaN
lignes_avec_nan = X_test['text_fr'].isna()

# Afficher les lignes avec NaN
print("\nLignes avec NaN dans la colonne 'texte':")
print(X_test.loc[lignes_avec_nan])


Lignes avec NaN dans la colonne 'texte':
Empty DataFrame
Columns: [productid, imageid, text, lang, imagefile, prdtypecode, text_fr]
Index: []


In [ ]:
X_train.to_csv("output/data-preprocessing-text-stopWord-steming/X_train_trad.csv")

In [ ]:
X_test.to_csv("output/data-preprocessing-text-stopWord-steming/X_test_trad.csv")